In [178]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import qmc
from pymoo.factory import get_problem

from selection import probability_of_selection, probability_of_dominance
from desdeo_tools.utilities import non_dominated


Implement probability of selection for single objective optimization. * (refer eqn.
12, 13, 14)

In [179]:
print(probability_of_selection([1, 2, 3], [0, 0, 0]))  # zero std
print(probability_of_selection([1, 2, 3], [2, 0.1, 0.1]))


[0.6666666666666666, 0.3333333333333333, 0.0]
[0.5204343109983712, 0.4308660522238991, 0.04869963677772956]


Implement probability of dominance and ranking. * (refer eqn. 15, 17, 18, 22)
- Generate 50 samples from DTLZ2 problem (3 objectives, 5 decision variables) using LHS sampling
- Build Kriging surrogates using the generated samples
- Resample or generate 20 data points randomly
- Rank the 20 points using probability of dominance ranking

In [180]:
# generate data
n_train = 50
n_test = 20

dtlz2 = get_problem("dtlz2", n_var=5, n_obj=3)
surrogates = [GaussianProcessRegressor() for _ in range(dtlz2.n_obj)]

lhs = qmc.LatinHypercube(dtlz2.n_var, seed=2)
X_train = lhs.random(n_train)
X_train = qmc.scale(X_train, dtlz2.xl, dtlz2.xu)
y_train = dtlz2.evaluate(X_train)
X_test = lhs.random(n_test)
X_test = qmc.scale(X_test, dtlz2.xl, dtlz2.xu)
y_test = dtlz2.evaluate(X_test)
# fit surrogates and predict
y_pred = np.zeros((n_test, dtlz2.n_obj))
y_pred_std = np.zeros((n_test, dtlz2.n_obj))
for surrogateI, surrogate in enumerate(surrogates):
    surrogate.fit(X_train, y_train[:, surrogateI])
    y_pred[:, surrogateI], y_pred_std[:, surrogateI] = surrogate.predict(
        X_test, return_std=True)


In [181]:
# print(probability_of_dominance(np.array([[0, 0], [1,1]]), np.array([[1,0], [0,0]])))
ps = probability_of_dominance(y_pred, y_pred_std)

np.set_printoptions(precision=2)
# sort based on probability of dominance
solutions = sorted(zip(X_test, y_pred, y_pred_std, ps, non_dominated(y_pred)),
                   key=lambda x: x[3], reverse=True)
print("values predicted, std,              probability,       nondominated")
for x, y, s, p, d in solutions:
    print(y, s, p, d)


values predicted, std,              probability,       nondominated
[0.28 0.19 0.98] [0.01 0.01 0.01] 0.05551533179499345 True
[0.06 0.07 1.17] [0.06 0.06 0.06] 0.05258092026540192 True
[1.09 0.23 0.05] [0.05 0.05 0.05] 0.05232291689859647 True
[0.89 0.04 0.83] [0.06 0.06 0.06] 0.05150568472473123 True
[0.38 0.76 0.87] [0.04 0.04 0.04] 0.050162776996306536 True
[0.85 0.45 0.69] [0.03 0.03 0.03] 0.05004601247405389 True
[0.8  0.88 0.29] [0.08 0.08 0.08] 0.05000449017379476 True
[0.3  1.15 0.34] [0.04 0.04 0.04] 0.05000329795530518 True
[0.04 1.28 0.55] [0.04 0.04 0.04] 0.04999914272038843 True
[0.64 0.85 0.65] [0.03 0.03 0.03] 0.04998055992754291 True
[-0.01  0.25  1.26] [0.08 0.08 0.08] 0.04992776736712933 True
[0.12 0.45 1.11] [0.03 0.03 0.03] 0.04984844173511168 True
[0.54 0.39 0.95] [0.03 0.03 0.03] 0.04975104237971577 True
[ 0.24 -0.01  1.33] [0.04 0.04 0.04] 0.04971522615261115 True
[0.2  0.27 1.14] [0.04 0.04 0.04] 0.04942491215201722 True
[1.06 0.43 0.46] [0.05 0.05 0.05] 0.0492